In [7]:
import pandas as pd
import requests
import numpy as np

In [8]:
df = pd.read_csv('../../raw_data/predict_set.csv',usecols=['date','company','title','sourceURL','body'],index_col='date',parse_dates=True).squeeze()

In [9]:
df

,sourceURL,company,title,body
date,,,,
2022-07-08,https://www.mycentraljersey.com/story/news/202...,Amazon,amazon port authority newark nj airport freigh...,Newark Airport deal for $432M Amazon air hub c...
2022-07-08,https://medcitynews.com/2022/07/amgen-joins-ke...,Amgen,amgen joins kernal bios 25m financing as the s...,A messenger RNA-based therapy won’t be very ef...
2022-07-08,https://www.agassizharrisonobserver.com/busine...,Apple,apple to add lockdown safeguard on iphones ipa...,Apple said it will roll out a “lockdown” optio...
2022-07-08,https://techstory.in/laid-off-tesla-employees-...,Tesla,laid off tesla employees find wor,Tesla laid off hundreds of its workforce recen...
2022-07-08,https://www.wateronline.com/doc/pepsico-s-new-...,PepsiCo,pepsico s new esg summary offers first look at...,PepsiCo's New ESG Summary Offers First Look At...
...,...,...,...,...
2022-10-09,https://tulsaworld.com/news/local/michael-over...,Starbucks,michael overall why did starbucks take so long...,"In the early 1990s, major renovations began on..."
2022-10-09,https://www.theepochtimes.com/rivian-electric-...,Amazon,rivian electric vehicle maker backed by amazon...,An electric vehicle maker backed by Amazon is ...
2022-10-09,https://www.forbes.com/sites/billybambrough/20...,KLA,get your money out right now elon musk leads i...,PayPal\nSubscribe now to Forbes' CryptoAsset &...


In [10]:
df.loc[df.index[0],'body'].values[0]

'Newark Airport deal for $432M Amazon air hub collapses\nAmazon and the Port Authority of New York and New Jersey have walked away from a controversial $432 million air freight hub proposed for Newark Liberty International Airport.\nWithout specifying what led to the breakdown, the Port Authority, which owns the airport, announced in an emailed statement Thursday that the two sides were parting ways after almost a year of talks.\n“Unfortunately, the Port Authority and Amazon have been unable to reach an agreement on final lease terms and mutually concluded that further negotiations will not resolve the outstanding issues,” Port Authority Chief Operating Officer Huntley Lawrence said in the statement.\nThe bistate agency and the e-commerce giant announced last August that they were negotiating a 20-year lease to repurpose two cargo buildings at Newark Airport. The project would create about 1,000 jobs and help speed shipping times for consumers across the region, they said.\nBut some la

In [11]:
for i in range(len(df)) :
    try:
        process_date = np.datetime64('now')
        company_response_get = requests.get(f"http://127.0.0.1:8000/companies/{df.loc[df.index[i],'company'].values[0]}")
        if company_response_get.status_code == 404:
            company_response = requests.post("http://127.0.0.1:8000/company/add/",json={'name':df.loc[df.index[i],'company'].values[0],
                                                                                    'description':'No description provided'})

        model_response = requests.post("http://127.0.0.1:8000/article/predict/",json={'body':df.loc[df.index[i],'body'].values[0]})
        response = requests.post(f"http://127.0.0.1:8000/article/add/{df.loc[df.index[i],'company'].values[0]}",json={
            "date": df.index[i].strftime('%Y-%m-%d'),
            "title": df.loc[df.index[i],'title'].values[0],
            "uploaded_at": pd.to_datetime(str(process_date)).strftime('%Y-%m-%d'),
            "body": df.loc[df.index[i],'body'].values[0],
            "sourceURL": df.loc[df.index[i],'sourceURL'].values[0],
            "topic_category": model_response.json()['topic_category'],
            "esg_score": model_response.json()['esg_score'],
            "scored_at": pd.to_datetime(str(process_date)).strftime('%Y-%m-%d'),
            "exclude_count": 0
        })
    except : 
        pass